In [ ]:
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use(r'~/PaperDoubleFig.mplstyle')

from scipy.integrate import solve_ivp


class XFEL:
    """
    One dimensional steady state FEL theory in universal scaling

    Class XFEL implements numerical integration of pendulum equations coupled
    with equation for the field A.
    """
    def __init__(self, A=0,
                 theta=0, d_theta=0.00186,
                 eta=0, d_eta=0,
                 n_theta=65, n_eta=19):
        """
        Setting parameters and initial conditions for an XFEL in universal
        scaling

        A - field source, complex amplitude of EM field in the case of
            amplifier config;
        theta - central phase for the ensembel of electrons;
        eta - central energy detuning for the ensemble of electrons;
        d_theta - 1st harmonic contribution to shot-noise $2/\sqrt{N_e}$;
        d_eta - energy detuning spread for the ensemble of electrons;

        n_theta - number of numerical electrons in a phase bucket;
        n_eta - number of numerical electrons in an energy distribution;

        n_eta*n_theta - total number of numerical electrons;

        d_theta>0 corresponds to A=dpsi/2 case
        d_theta = 0.00138 corresponds to the MaRIE case
        A = 0.00069 corresponds to the shot-noise power in the MaRIE case
        """

        self.k0 = eta
        self.psi = theta
        self.dk0 = d_eta
        self.dpsi = d_theta
        self._A0 = A

        self._npsi = n_theta  # 8 should be sufficient
        self._nk = n_eta  # 3 should be sufficient
        self.ne = self._npsi * self._nk  # total number of numerical particles

        # quiet start implementation according to H Freund
        # 1. the Gaussian Quadrature approach is used here for the phase
        xpsi, wpsi = np.polynomial.legendre.leggauss(self._npsi)

        wpsi = self._npsi * wpsi / 2.  # normalization
        psi0 = self.psi + np.pi * xpsi  # quite start positions
        psi0 -= self.dpsi * np.sin(psi0)  # shot-noise shift @ 1st Harmonic

        # 2. the Gauss-Hermite quadrature generation is used for energy spread
        xk, wk = np.polynomial.hermite.hermgauss(self._nk)
        wk = self._nk * wk / np.sqrt(np.pi)  # normalization
        k0 = self.k0 + np.sqrt(2) * self.dk0 * xk  # initial positions

        # creating arrays of the particles and weights
        psi0, k0 = np.meshgrid(psi0, k0)  # combining all numerical particles
        self._theta0, self._eta0 = psi0.flatten(), k0.flatten()  # creating arrays
        wpsi, wk = np.meshgrid(wpsi, wk)  # combinig weights of the particles
        self.charge = (wpsi * wk).flatten()  # creating array of weights

    def change_start_values(self, A0, theta0, eta0):
        assert len(theta0) == self.ne
        assert len(eta0) == self.ne
        self._A0 = A0
        self._theta0 = theta0
        self._eta0 = eta0
        print("Distribution is equally weighted now.")
        self.charge = np.ones(self.ne)
        

    def model(self, tstop, dt=0.1):  # 0.1 step seems to be accurate enough
        def rhs(t, y):
            """
            The right-hand side of the 1D canonical FEL equations;
            t - the current time;
            y - array of [A, theta, eta]
            """
            A = y[0]
            theta = y[1:self.ne+1]
            eta = y[self.ne+1:]
            dA_dt = np.mean(self.charge*np.exp(-1j*theta))
            dtheta_dt = eta
            deta_dt = -2*np.real(A*np.exp(1j*theta))
            return np.concatenate(([dA_dt],
                                   dtheta_dt,
                                   deta_dt))
        self.tstop = tstop
        self._sol = solve_ivp(rhs, [0, tstop],
                              np.concatenate(([self._A0+0j],
                                              self._theta0,
                                              self._eta0)),
                              max_step=dt,
                              dense_output=True
                             )

    def A(self, t):
        return self._sol.sol(t)[0]

    def theta(self, t):
        return np.real(self._sol.sol(t)[1:self.ne+1]+np.pi)%(2*np.pi)-np.pi
        #return np.real(self._sol.sol(t)[1:self.ne+1])

    def eta(self, t):
        return np.real(self._sol.sol(t)[self.ne+1:])
    
    def show_evolution(self):
        phase = np.unwrap(np.angle(self._sol.y[0]))
        power = np.abs(self._sol.y[0])**2
        t = self._sol.t
        fig1 = plt.figure()

        ax1L = plt.subplot(111)  # creates an active axes object
        ax1L.set_xlabel(r'Dimensionless time, $\tau$')

        plt.axis([0, self.tstop, np.min(phase), np.max(phase)])  # sets limits on an active object
        ax1L.set_ylabel('Phase', color='b')
        for tl in ax1L.get_yticklabels():
            tl.set_color('b')

        ax1R = ax1L.twinx()  # adds a twin y axis using same x

        plt.axis([0, self.tstop, 0, np.max(power)])  # sets limits on an active object
        ax1R.set_ylabel(r'Dimensionless power, $|A|^2$', color='r')
        for tl in ax1R.get_yticklabels():
            tl.set_color('r')

        line1, = ax1L.plot([], [], 'b')  # creates blue line on left axis
        line2, = ax1R.plot([], [], 'r')  # creates red line on right axis
        line1.set_data(t, phase)
        line2.set_data(t, power)

        plt.show()
        
    def show_instance(self, t):
        assert 0<=t<=self.tstop
        #  Let us plot phase space and separatrix using scatter plot function
        fig2 = plt.figure()
        ax2L = plt.subplot(111)  # creates an active axes object below first
        ax2L.set_xlabel(r'Ponderomotive phase, $\theta_n$')

        ax2L.axis([-np.pi, np.pi, -4, 3])  # limits set
        ax2L.set_ylabel('Electron\'s detuning, $\eta_n$', color='b')
        for tl in ax2L.get_yticklabels():
            tl.set_color('b')
        ax2L.scatter(self.theta(t), self.eta(t), 20*laser.charge, marker=".", color='b')

        ax2R = ax2L.twinx()  # adds a twin y axis using same x
        ax2R.set_ylim([-4, 3])  # limits set

        ax2R.set_ylabel(r'Separatrix', color='r')
        for tl in ax2R.get_yticklabels():
            tl.set_color('r')

        x = np.linspace(-np.pi, np.pi, 500)
        ax2R.plot(x,
                 -2*np.sqrt(np.abs(np.imag(self.A(t)*np.exp(1j*x)))),
                 'r--', linewidth=2.0)
        ax2R.plot(x,
                 2*np.sqrt(np.abs(np.imag(self.A(t)*np.exp(1j*x)))),
                 'r--', linewidth=2.0)
        plt.show()

        

if __name__ == '__main__':
    tstop = 4*np.pi
    laser = XFEL(d_eta=0.1)
    laser.model(tstop)
    laser.show_evolution()
    laser.show_instance(9.5)

In [2]:
from ipywidgets import interactive, FloatSlider, Play

dt = 0.1
def f(t):
    laser.show_instance(dt*t)

interactive(f, t=Play(min=0, max=tstop/dt))

interactive(children=(Play(value=0, description='t', max=125), Output()), _dom_classes=('widget-interact',))

In [3]:
def f1(t):
    laser.show_instance(t)

interactive(f1, t=FloatSlider(min=0, max=tstop, step=dt, value=2.0))

interactive(children=(FloatSlider(value=2.0, description='t', max=12.566370614359172), Output()), _dom_classes…